In [1]:
import json
import sys
import numpy as np
import pandas as pd

userdata = json.loads('{\"USER_CODE\":39,\"USER_TASTE\":\"0,22\"}')
postdata = json.loads('[{\"NOTICEBOARD_NUM\":8,\"LABELS\":\"3,25\"},{\"NOTICEBOARD_NUM\":9,\"LABELS\":\"4,17,22\"},{\"NOTICEBOARD_NUM\":10,\"LABELS\":\"3,10,15,17\"},{\"NOTICEBOARD_NUM\":11,\"LABELS\":\"1,23\"},{\"NOTICEBOARD_NUM\":12,\"LABELS\":\"1,17,21\"},{\"NOTICEBOARD_NUM\":13,\"LABELS\":\"2\"},{\"NOTICEBOARD_NUM\":14,\"LABELS\":\"17,21\"},{\"NOTICEBOARD_NUM\":15,\"LABELS\":\"21\"},{\"NOTICEBOARD_NUM\":16,\"LABELS\":\"1\"}]')

label_leng = 29
print(type(postdata))

post_df = pd.DataFrame(postdata)
print(post_df['NOTICEBOARD_NUM'])
post_df

<class 'list'>
0     8
1     9
2    10
3    11
4    12
5    13
6    14
7    15
8    16
Name: NOTICEBOARD_NUM, dtype: int64


,NOTICEBOARD_NUM,LABELS
0,8,"3,25"
1,9,"4,17,22"
2,10,"3,10,15,17"
3,11,"1,23"
4,12,"1,17,21"
5,13,2
6,14,"17,21"
7,15,21
8,16,1


In [3]:
temp = np.zeros(label_leng)
if post_df.loc[0]["LABELS"] is not None:
    for ch in post_df.loc[0]["LABELS"].split(',') :
        temp[int(ch)-1] = 1

num_by_label = pd.DataFrame({post_df.loc[0]["NOTICEBOARD_NUM"] : temp})

for i in range(1, int(post_df.size/2)):
    temp = np.zeros(label_leng)
    if post_df.loc[i]["LABELS"] is not None:
        for ch in post_df.loc[i]["LABELS"].split(',') :
            temp[int(ch)-1] = 1
    temp_df = pd.DataFrame({post_df.loc[i]["NOTICEBOARD_NUM"] : temp})

    num_by_label = pd.concat([num_by_label, temp_df], axis=1)

num_by_label

,8,9,10,11,12,13,14,15,16
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
user_taste_row = np.zeros(29)
user_taste = userdata['USER_TASTE'].split(',')
user_taste = [int(taste) for taste in user_taste]

for i in user_taste:
    user_taste_row[i] = 1

user_taste_df = pd.DataFrame({userdata['USER_CODE'] : user_taste_row})
user_taste_df = user_taste_df.T
user_taste_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
39,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
result = user_taste_df.dot(num_by_label)
result

,8,9,10,11,12,13,14,15,16
39,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0


In [10]:
result/len(user_taste)

,8,9,10,11,12,13,14,15,16
39,0.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.5


In [11]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)